In [3]:
import pandas as pd
import glob
from datetime import datetime
import time

#### Preprocessing of the raw collected data into the table where one row shows one period of time that the bike spent on the same location (station)

In [13]:
url = "output_datasets/cleaned_data_nextbike_2020-02.csv.gz"
df = pd.read_csv(url, sep=';', low_memory=False)

In [14]:
fmt = '%Y-%m-%d %H:%M:%S'

In [15]:
table = df.head()

In [16]:
table

lat       lng                  time_begin  \
0  50.938699  6.934269  2020-02-01 02:53:02.381322   
1  50.938699  6.934264  2020-02-01 02:54:02.716220   
2  50.950791  6.922121  2020-02-01 03:51:02.284672   
3  50.950787  6.922128  2020-02-01 03:52:02.712139   
4  50.927378  6.939596  2020-02-01 19:41:03.036455   

                     time_end        name address  bike  bike_racks  bikes  \
0  2020-02-01 02:53:02.381322  BIKE 21264     NaN  True           0      1   
1  2020-02-01 03:27:02.394296  BIKE 21264     NaN  True           0      1   
2  2020-02-01 03:51:02.284672  BIKE 21264     NaN  True           0      1   
3  2020-02-01 19:17:01.915981  BIKE 21264     NaN  True           0      1   
4  2020-02-01 19:41:03.036455  BIKE 21264     NaN  True           0      1   

   booked_bikes  ...  rack_locks  special_racks   spot  terminal_type  \
0             0  ...       False              0  False            NaN   
1             0  ...       False              0  False            NaN   
2             0  ...       False              0  False            NaN   
3             0  ...       False              0  False            NaN   
4             0  ...       False              0  False            NaN   

        uid  city  country_code   company       timezone  available_bikes  
0  27455312  Köln            DE  Nextbike  Europe/Berlin             2660  
1  27455312  Köln            DE  Nextbike  Europe/Berlin             2660  
2  27458309  Köln            DE  Nextbike  Europe/Berlin             2659  
3  27458309  Köln            DE  Nextbike  Europe/Berlin             2660  
4  27473030  Köln            DE  Nextbike  Europe/Berlin             2626  

[5 rows x 25 columns]

In [17]:
table.drop(['address', 'bike', 'bike_racks', 'bikes', 'booked_bikes', 'free_racks', 'free_special_racks', 'maintenance', 'number',
            'place_type', 'rack_locks', 'special_racks', 'spot', 'terminal_type', 'city',
            'country_code', 'company', 'timezone'],axis=1, inplace=True)

C:\Users\nadja\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [18]:
table

lat       lng                  time_begin  \
0  50.938699  6.934269  2020-02-01 02:53:02.381322   
1  50.938699  6.934264  2020-02-01 02:54:02.716220   
2  50.950791  6.922121  2020-02-01 03:51:02.284672   
3  50.950787  6.922128  2020-02-01 03:52:02.712139   
4  50.927378  6.939596  2020-02-01 19:41:03.036455   

                     time_end        name       uid  available_bikes  
0  2020-02-01 02:53:02.381322  BIKE 21264  27455312             2660  
1  2020-02-01 03:27:02.394296  BIKE 21264  27455312             2660  
2  2020-02-01 03:51:02.284672  BIKE 21264  27458309             2659  
3  2020-02-01 19:17:01.915981  BIKE 21264  27458309             2660  
4  2020-02-01 19:41:03.036455  BIKE 21264  27473030             2626

In [19]:
with open('mytable.tex','w') as tf:
    tf.write(table.to_latex())

In [ ]:
new_df = pd.DataFrame(columns= ['lat', 'lng', 'time_begin', 'time_end', 'standing_time_minutes', 'uid', 'name', 'address', 'bike', 'bike_racks', 'bikes', 
                                'booked_bikes', 'free_racks', 'free_special_racks', 'maintenance', 'number', 
                                'place_type', 'rack_locks', 'special_racks', 'spot', 'terminal_type', 'city',
                                'country_code', 'company', 'timezone', 'available_bikes'] ) #empty dataframe with the new column structure

In [ ]:
for index, row in df.iterrows():
    t_begin = df['time_begin'][index].split('.')[0]
    t_end = df['time_end'][index].split('.')[0]    
    
    d1 = datetime.strptime(t_begin, fmt)
    d2 = datetime.strptime(t_end, fmt)
    
    d1_ts = time.mktime(d1.timetuple())
    d2_ts = time.mktime(d2.timetuple())
    
    minsDiff = round((int(d2_ts-d1_ts) / 60), 0)
    new_df = new_df.append({'lat' : row['lat'], 'lng' : row['lng'] ,
                            'time_begin' : row['time_begin'], 'time_end' : row['time_end'], 
                            'standing_time_minutes' : minsDiff, 'uid' : row['uid'], 'address' : row['address'], 
                            'bike' : row['bike'], 'bike_racks' : row['bike_racks'], 'bikes' : row['bikes'], 
                            'booked_bikes' : row['booked_bikes'], 'free_racks' : row['free_racks'], 
                            'free_special_racks' : row['free_special_racks'], 
                            'maintenance' : row['maintenance'], 'name' : row['name'], 'number' : row['number'], 
                            'place_type' : row['place_type'], 'rack_locks' : row['rack_locks'], 
                            'special_racks' : row['special_racks'], 'spot' : row['spot'], 
                            'terminal_type' : row['terminal_type'], 'city' : row['city'], 
                            'country_code' : row['country_code'], 'company' : row['company'], 
                            'timezone' : row['timezone'], 'available_bikes' : row['available_bikes']}, ignore_index=True)

In [ ]:
new_df

In [ ]:
new_df.to_csv('time_on_station_data_nextbike_2019-12.csv', sep=';', index=False)